# Zoho2CSV

Get the email from and subjectl liens for my filters

go to: 
https://api-console.zoho.com/

create a config.json and add the appripriate information:

```json
{
  "Echo": "Working",
  "client_id": "your_client_id",
  "client_secret": "your_client_secret",
  "redirect_uri": "your_redirect_uri",
  "scope": "ZohoMail.accounts.READ",
  "refresh_token": ""
}
```

## client - server

Need a redirect URL

In [ ]:
import requests
import json
import webbrowser

# Load sensitive information from config.json
def load_config():
    with open("config.json", "r") as file:
        return json.load(file)



In [ ]:
# Load sensitive information from config.json
def load_config():
    with open("config.json", "r") as file:
        return json.load(file)


In [ ]:

# Save access and refresh tokens to config.json
def save_tokens(tokens):
    with open("config.json", "r+") as file:
        data = json.load(file)
        data['access_token'] = tokens['access_token']
        data['refresh_token'] = tokens['refresh_token']
        file.seek(0)
        json.dump(data, file, indent=4)


In [ ]:
def get_authorization_code():
    config = load_config()  # Ensure config.json has the correct client_id
    print (config['Echo'])
    auth_url = (
        f"https://accounts.zoho.com/oauth/v2/auth"
        f"?response_type=code"
        f"&client_id={config['client_id']}"
        f"&scope={config['scope']}"
        f"&redirect_uri={config['redirect_uri']}"
        f"&access_type=offline"
    )
    print(f"Open this URL to authorize the app:\n{auth_url}")
    webbrowser.open(auth_url)
    authorization_code = input("Enter the authorization code from the URL: ")
    return authorization_code


In [ ]:
# Step 2: Exchange authorization code for access token and refresh token
def get_tokens(authorization_code):
    config = load_config()
    token_url = "https://accounts.zoho.com/oauth/v2/token"
    payload = {
        "code": authorization_code,
        "grant_type": "authorization_code",
        "client_id": config['client_id'],
        "client_secret": config['client_secret'],
        "redirect_uri": config['redirect_uri']
    }
    response = requests.post(token_url, data=payload)
    if response.status_code == 200:
        tokens = response.json()
        save_tokens(tokens)
        print("Access and refresh tokens saved successfully.")
    else:
        print("Failed to get tokens:", response.text)


In [ ]:
# Step 3: Make API requests using the access token
def make_api_request():
    config = load_config()
    api_url = "https://mail.zoho.com/api/accounts"
    headers = {
        "Authorization": f"Zoho-oauthtoken {config['access_token']}"
    }
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        print("API request successful!")
        print(response.json())
    else:
        print("API request failed:", response.text)


In [ ]:
# Refresh the access token when expired
def refresh_access_token():
    config = load_config()
    token_url = "https://accounts.zoho.com/oauth/v2/token"
    payload = {
        "refresh_token": config['refresh_token'],
        "grant_type": "refresh_token",
        "client_id": config['client_id'],
        "client_secret": config['client_secret']
    }
    response = requests.post(token_url, data=payload)
    if response.status_code == 200:
        tokens = response.json()
        save_tokens(tokens)
        print("Access token refreshed successfully.")
    else:
        print("Failed to refresh access token:", response.text)



In [ ]:
# Main flow
if __name__ == "__main__":
    # Step 1: Get authorization code manually by visiting the URL
    authorization_code = get_authorization_code()



In [ ]:
    # Step 2: Exchange authorization code for tokens
    get_tokens(authorization_code)


In [ ]:
    # Step 3: Use the access token to make API requests
    make_api_request()

## self client

In [1]:
import requests
import json

# Load sensitive information from config.json
def load_config():
    with open("config.json", "r") as file:
        return json.load(file)

# Save access and refresh tokens to config.json
def save_tokens(tokens):
    with open("config.json", "r+") as file:
        data = json.load(file)
        data['access_token'] = tokens['access_token']
        data['refresh_token'] = tokens['refresh_token']
        file.seek(0)
        json.dump(data, file, indent=4)




In [2]:
def get_tokens(authorization_code):
    config = load_config()
    token_url = "https://accounts.zoho.com/oauth/v2/token"
    payload = {
        "code": authorization_code,
        "grant_type": "authorization_code",
        "client_id": config['client_id'],
        "client_secret": config['client_secret'],
        "redirect_uri": config['redirect_uri']  # This can be a static value if not needed
    }
    response = requests.post(token_url, data=payload)
    print("Response Status Code:", response.status_code)
    print("Response Content:", response.text)  # Print full response for debugging
    
    if response.status_code == 200:
        tokens = response.json()
        if 'access_token' in tokens:
            save_tokens(tokens)
            print("Access and refresh tokens saved successfully.")
        else:
            print("Access token not found in the response. Full response:", tokens)
    else:
        print("Failed to get tokens:", response.text)


In [16]:
# Main flow
if __name__ == "__main__":
    authorization_code = input("Enter the authorization code from Zoho Console: ")
    get_tokens(authorization_code)

Enter the authorization code from Zoho Console:  1000.4f69851f72656cde3d458fe95f748e40.afcdad62ab391176b8ef551ae463bce8


Response Status Code: 200
Response Content: {"access_token":"1000.315b93c1a40634aa58c27749b7468f9f.f39b539e581534f795d0ea2cb60cca22","refresh_token":"1000.bdfc2fef998f0f24447bc28406675867.a72ece486e03e51cdf561731c81636d1","scope":"ZohoMail.accounts.READ ZohoMail.folders.READ ZohoMail.messages.READ","api_domain":"https://www.zohoapis.com","token_type":"Bearer","expires_in":3600}
Access and refresh tokens saved successfully.


## Make API request

In [3]:
def make_api_request():
    config = load_config()
    api_url = "https://mail.zoho.com/api/accounts"  # This endpoint retrieves your mail accounts
    headers = {
        "Authorization": f"Zoho-oauthtoken {config['access_token']}"
    }
    
    # Make the API request
    response = requests.get(api_url, headers=headers)
    
    if response.status_code == 200:
        print("API request successful!")
        print(json.dumps(response.json(), indent=4)) # Print the response to see your mail accounts or other details
    else:
        print("API request failed:", response.text)

In [ ]:
make_api_request()

In [5]:
account_id = input("Enter the account_id from Zoho Console: ")

Enter the account_id from Zoho Console:  1044620000000008003


In [6]:
def get_folders():
    config = load_config()
    api_url = f"https://mail.zoho.com/api/accounts/{account_id}/folders"  # Replace {account_id} with your actual account ID
    headers = {
        "Authorization": f"Zoho-oauthtoken {config['access_token']}"
    }
    
    response = requests.get(api_url, headers=headers)
    
    if response.status_code == 200:
        folders = response.json()
        # Print all folders and find the Inbox
        for folder in folders['data']:
            print(f"Folder Name: {folder['folderName']}, Folder ID: {folder['folderId']}")
    else:
        print("Failed to fetch folders:", response.text)

In [7]:
get_folders()

Folder Name: Inbox, Folder ID: 1044620000000008013
Folder Name: Drafts, Folder ID: 1044620000000008015
Folder Name: Templates, Folder ID: 1044620000000008017
Folder Name: Sent, Folder ID: 1044620000000008019
Folder Name: Spam, Folder ID: 1044620000000008021
Folder Name: Trash, Folder ID: 1044620000000008023
Folder Name: Outbox, Folder ID: 1044620000000008025
Folder Name: personal, Folder ID: 1044620000000019003
Folder Name: Sent Messages, Folder ID: 1044620000000322001
Folder Name: Deleted Messages, Folder ID: 1044620000000348001
Folder Name: Shipping, Folder ID: 1044620000001466001
Folder Name: Business, Folder ID: 1044620000001793003
Folder Name: Finance, Folder ID: 1044620000009125143
Folder Name: Medical License, Folder ID: 1044620000009125145
Folder Name: ASCP, Folder ID: 1044620000009125147
Folder Name: Chase, Folder ID: 1044620000009125158
Folder Name: PayPal, Folder ID: 1044620000009125169
Folder Name: Bills, Folder ID: 1044620000009125191
Folder Name: Military, Folder ID: 1044

In [8]:
folder_id = input("Enter the folder_id from Zoho Console: ")

Enter the folder_id from Zoho Console:  1044620000000008013


In [9]:
def get_inbox_messages(account_id, folder_id):
    config = load_config()
    api_url = f"https://mail.zoho.com/api/accounts/{account_id}/messages/view"
    headers = {
        "Authorization": f"Zoho-oauthtoken {config['access_token']}"
    }

    # Query parameters for the inbox messages
    params = {
        "folderId": folder_id,
        "start": 1,       # Start from the first email
        "limit": 10,      # Retrieve 10 emails
        "status": "all",  # Retrieve all emails (read and unread)
        "threadedMails": "false"  # Do not retrieve threaded emails
    }

    response = requests.get(api_url, headers=headers, params=params)

    if response.status_code == 200:
        emails = response.json()['data']
        for email in emails:
            # Extract 'fromAddress' and 'subject' fields
            from_address = email.get('fromAddress', 'Unknown sender')
            subject = email.get('subject', 'No subject')
            print(f"From: {from_address}, Subject: {subject}")
    else:
        print("Failed to fetch inbox emails:", response.text)


In [10]:
get_inbox_messages(account_id, folder_id)

From: no-reply@worldcue.worldaware.com, Subject: Warn Alert - MENA - Protests, Increased Security - Civil Unrest
From: no-reply@worldcue.worldaware.com, Subject: Info Alert - Iraq - Mourning Period - Nationwide - Civil Unrest
From: info@fileflorida.com, Subject: PAST DUE - ROCKLEDGE HIGH SCHOOL CLASS OF 2012 ALUMNI INC - 2024 Annual Report Filing from File Florida Co.
From: do-not-reply@sonyb2c.onmicrosoft.com, Subject: Mã OTP xác nhận của My Sony Rewards
From: do-not-reply@sonyb2c.onmicrosoft.com, Subject: Mã OTP xác nhận của My Sony Rewards
From: eticket@amadeus.com, Subject: Your Electronic Receipt
From: no-reply-vietcombank@nganluong.vn, Subject: Payment Confirmation E-Visa for register code E240920USAA3523506278
From: e-invoice@phongvu.vn, Subject: Email hóa đơn điện tử
From: noreply@vnticketonline.vn, Subject: [ĐSVN] Thông báo mua vé thành công
From: zengoteam@hello.zengo.com, Subject: Enjoy 10% off Bankless Citizenship with Zengo Pro


In [13]:
import pandas as pd

def save_inbox_messages_to_excel(account_id, folder_id):
    config = load_config()
    api_url = f"https://mail.zoho.com/api/accounts/{account_id}/messages/view"
    headers = {
        "Authorization": f"Zoho-oauthtoken {config['access_token']}"
    }

    params = {
        "folderId": folder_id,
        "start": 1,
        "limit": 200,
        "status": "all",
        "threadedMails": "false"
    }

    response = requests.get(api_url, headers=headers, params=params)

    if response.status_code == 200:
        emails = response.json()['data']
        # Extract email data
        email_data = [{"From": email.get('fromAddress', 'Unknown sender'), 
                       "Subject": email.get('subject', 'No subject')} for email in emails]
        
        # Convert to DataFrame and save to Excel
        df = pd.DataFrame(email_data)
        df.to_excel('inbox_emails.xlsx', index=False)
        print("Inbox emails saved to inbox_emails.xlsx")
    else:
        print("Failed to fetch inbox emails:", response.text)


In [14]:
save_inbox_messages_to_excel(account_id, folder_id)

Inbox emails saved to inbox_emails.xlsx


## Batch get and organize

In [15]:
import pandas as pd
import time
import re

In [16]:
# Function to fetch emails in batches of 200 with a timeout
def fetch_emails_in_batches(account_id, folder_id, total_emails, batch_size=200, timeout=2):
    config = load_config()
    api_url = f"https://mail.zoho.com/api/accounts/{account_id}/messages/view"
    headers = {
        "Authorization": f"Zoho-oauthtoken {config['access_token']}"
    }

    all_emails = []

    for start in range(1, total_emails, batch_size):
        # Query parameters for the inbox messages
        params = {
            "folderId": folder_id,
            "start": start,
            "limit": batch_size,
            "status": "all",
            "threadedMails": "false"
        }

        response = requests.get(api_url, headers=headers, params=params)

        if response.status_code == 200:
            emails = response.json()['data']
            all_emails.extend(emails)
            print(f"Fetched {len(emails)} emails starting from {start}")
        else:
            print(f"Failed to fetch emails starting from {start}: {response.text}")
            break  # Exit if there's an error fetching a batch

        # Timeout between requests
        time.sleep(timeout)

    # Convert the list of emails to a DataFrame
    email_data = [{"From": email.get('fromAddress', 'Unknown sender'), 
                   "Subject": email.get('subject', 'No subject')} for email in all_emails]
    
    df = pd.DataFrame(email_data)
    return df

In [18]:
total_emails = 1300             # Total number of emails to fetch

# Step 1: Fetch all emails in batches and store in a DataFrame
email_df = fetch_emails_in_batches(account_id, folder_id, total_emails)

Fetched 200 emails starting from 1
Fetched 200 emails starting from 201
Fetched 200 emails starting from 401
Fetched 200 emails starting from 601
Fetched 200 emails starting from 801
Fetched 200 emails starting from 1001
Fetched 48 emails starting from 1201


In [19]:
# Function to extract domain from email
def extract_domain(email_address):
    try:
        domain = re.search("@[\w.]+", email_address).group()
        return domain
    except AttributeError:
        return "Unknown"


In [17]:
# Function to desensitize the data by extracting unique domains
def desensitize_email_domains(df):
    # Extract the domain from the 'From' email address
    df['Domain'] = df['From'].apply(extract_domain)

    # Get the unique domains
    unique_domains_df = df[['Domain']].drop_duplicates().reset_index(drop=True)
    
    return unique_domains_df



In [20]:
# Step 2: Desensitize the data by extracting unique domains
domain_df = desensitize_email_domains(email_df)

In [21]:
# Display the unique domains
domain_df.head()

,Domain
0,@worldcue.worldaware.com
1,@fileflorida.com
2,@sonyb2c.onmicrosoft.com
3,@amadeus.com
4,@nganluong.vn


In [22]:
# Optionally save the domains to an Excel file
domain_df.to_excel('unique_email_domains.xlsx', index=False)
print("Unique email domains saved to unique_email_domains.xlsx")

Unique email domains saved to unique_email_domains.xlsx


In [23]:
email_df.head()

,From,Subject,Domain
0,no-reply@worldcue.worldaware.com,"Warn Alert - MENA - Protests, Increased Securi...",@worldcue.worldaware.com
1,no-reply@worldcue.worldaware.com,Info Alert - Iraq - Mourning Period - Nationwi...,@worldcue.worldaware.com
2,info@fileflorida.com,PAST DUE - ROCKLEDGE HIGH SCHOOL CLASS OF 2012...,@fileflorida.com
3,do-not-reply@sonyb2c.onmicrosoft.com,Mã OTP xác nhận của My Sony Rewards,@sonyb2c.onmicrosoft.com
4,do-not-reply@sonyb2c.onmicrosoft.com,Mã OTP xác nhận của My Sony Rewards,@sonyb2c.onmicrosoft.com


In [24]:
email_df.to_excel('inbox_emails.xlsx', index=False)
print("Unique email domains saved to inbox_emails")

Unique email domains saved to inbox_emails


## Categorize

In [ ]:
# Defining general keywords for each category based on common associations
category_keywords = {
    'Document': ['document', 'file', 'attachment'],
    'Work': ['project', 'meeting', 'deadline', 'work'],
    'Ads_business': ['promotion', 'sale', 'offer', 'ad' "%" ],
    'Education': ['school', 'class', 'education', 'course'],
    'Active': ['active', 'account', 'status'],
    'Health': ['health', 'wellness', 'fitness', 'doctor'],
    'ToMe': ['personal', 'for you'],
    'Games': ['game', 'gaming'],
    'Travel': ['travel', 'flight', 'trip', 'hotel', 'Iraq', 'MENA'],
    'Tax': ['tax', 'IRS', 'refund'],
    'Gov': ['government', 'gov', 'official'],
    'EbenefitUSAA': ['ebenefit', 'usaa', 'veteran'],
    'Retirement': ['retirement', 'pension', '401k'],
    'Crypto': ['crypto', 'bitcoin', 'ethereum'],
    'Schwab': ['schwab', 'investment'],
    'Jobs': ['job', 'career', 'hiring'],
    'News': ['news', 'headline', 'report'],
    'Bills': ['bill', 'invoice', 'payment due'],
    'Shipping': ['shipping', 'delivery', 'tracking'],
    'Other Miitary': ['military', 'defense', 'army', 'navy'],
    'Oter Finance': ['finance', 'bank', 'loan', 'mortgage'],
    'Stocks': ['stocks', 'shares', 'market'],
    'ads': ['ad', 'advertisement', 'promotion'],
    'Paypal': ['paypal'],
    'Chase': ['chase', 'bank'],
    'ASCP': ['ascp', 'certification'],
    'Personal': ['personal', 'message', 'greetings']
}

# Function to categorize based on subject line
def categorize_email(subject):
    for category, keywords in category_keywords.items():
        if any(keyword.lower() in str(subject).lower() for keyword in keywords):
            return category
    return 'Uncategorized'

# Applying the function to categorize each email based on subject
email_df['Category'] = email_df['Subject'].apply(categorize_email)

## automated: 

In [ ]:
def get_account_id():
    config = load_config()
    api_url = "https://mail.zoho.com/api/accounts"
    headers = {
        "Authorization": f"Zoho-oauthtoken {config['access_token']}"
    }

    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        accounts = response.json()
        for account in accounts['data']:
            print(f"Account ID: {account['accountId']}, Email Address: {account['emailAddress']}")
        return accounts['data'][0]['accountId']  # Return the first account ID if multiple accounts exist
    else:
        print("Failed to fetch accounts:", response.text)


In [ ]:
def get_folders(account_id):
    config = load_config()
    api_url = f"https://mail.zoho.com/api/accounts/{account_id}/folders"
    headers = {
        "Authorization": f"Zoho-oauthtoken {config['access_token']}"
    }

    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        folders = response.json()
        for folder in folders['data']:
            print(f"Folder Name: {folder['folderName']}, Folder ID: {folder['folderId']}")
        return folders
    else:
        print("Failed to fetch folders:", response.text)


In [ ]:
def get_inbox_messages(account_id, folder_id):
    config = load_config()
    api_url = f"https://mail.zoho.com/api/accounts/{account_id}/messages/view"
    headers = {
        "Authorization": f"Zoho-oauthtoken {config['access_token']}"
    }

    # Query parameters for the inbox messages
    params = {
        "folderId": folder_id,
        "start": 1,       # Start from the first email
        "limit": 10,      # Retrieve 10 emails
        "status": "all",  # Retrieve all emails (read and unread)
        "threadedMails": "false"  # Do not retrieve threaded emails
    }

    response = requests.get(api_url, headers=headers, params=params)

    if response.status_code == 200:
        emails = response.json()
        for email in emails['data']:
            print(f"From: {email['from']['address']}, Subject: {email['subject']}")
        return emails
    else:
        print("Failed to fetch inbox emails:", response.text)


In [ ]:

# Step 1: Get the account ID
account_id = get_account_id()

# Step 2: Get the folder ID for Inbox (look for the folder named 'Inbox')
folders = get_folders(account_id)
inbox_folder_id = None
for folder in folders['data']:
    if folder['folderName'].lower() == 'inbox':
        inbox_folder_id = folder['folderId']
        break

if inbox_folder_id:
    # Step 3: Fetch and display emails from Inbox
    get_inbox_messages(account_id, inbox_folder_id)
else:
    print("Inbox folder not found.")
